
<br>
====================================================<br>
Comparison of segmentation and superpixel algorithms<br>
====================================================<br>
This example compares four popular low-level image segmentation methods.  As<br>
it is difficult to obtain good segmentations, and the definition of "good"<br>
often depends on the application, these methods are usually used for obtaining<br>
an oversegmentation, also known as superpixels. These superpixels then serve as<br>
a basis for more sophisticated algorithms such as conditional random fields<br>
(CRF).<br>
Felzenszwalb's efficient graph based segmentation<br>
-------------------------------------------------<br>
This fast 2D image segmentation algorithm, proposed in [1]_ is popular in the<br>
computer vision community.<br>
The algorithm has a single ``scale`` parameter that influences the segment<br>
size. The actual size and number of segments can vary greatly, depending on<br>
local contrast.<br>
.. [1] Efficient graph-based image segmentation, Felzenszwalb, P.F. and<br>
       Huttenlocher, D.P.  International Journal of Computer Vision, 2004<br>
Quickshift image segmentation<br>
-----------------------------<br>
Quickshift is a relatively recent 2D image segmentation algorithm, based on an<br>
approximation of kernelized mean-shift. Therefore it belongs to the family of<br>
local mode-seeking algorithms and is applied to the 5D space consisting of<br>
color information and image location [2]_.<br>
One of the benefits of quickshift is that it actually computes a<br>
hierarchical segmentation on multiple scales simultaneously.<br>
Quickshift has two main parameters: ``sigma`` controls the scale of the local<br>
density approximation, ``max_dist`` selects a level in the hierarchical<br>
segmentation that is produced. There is also a trade-off between distance in<br>
color-space and distance in image-space, given by ``ratio``.<br>
.. [2] Quick shift and kernel methods for mode seeking,<br>
       Vedaldi, A. and Soatto, S.<br>
       European Conference on Computer Vision, 2008<br>
SLIC - K-Means based image segmentation<br>
---------------------------------------<br>
This algorithm simply performs K-means in the 5d space of color information and<br>
image location and is therefore closely related to quickshift. As the<br>
clustering method is simpler, it is very efficient. It is essential for this<br>
algorithm to work in Lab color space to obtain good results.  The algorithm<br>
quickly gained momentum and is now widely used. See [3]_ for details.  The<br>
``compactness`` parameter trades off color-similarity and proximity, as in the<br>
case of Quickshift, while ``n_segments`` chooses the number of centers for<br>
kmeans.<br>
.. [3] Radhakrishna Achanta, Appu Shaji, Kevin Smith, Aurelien Lucchi,<br>
    Pascal Fua, and Sabine Suesstrunk, SLIC Superpixels Compared to<br>
    State-of-the-art Superpixel Methods, TPAMI, May 2012.<br>
Compact watershed segmentation of gradient images<br>
-------------------------------------------------<br>
Instead of taking a color image as input, watershed requires a grayscale<br>
*gradient* image, where bright pixels denote a boundary between regions.<br>
The algorithm views the image as a landscape, with bright pixels forming high<br>
peaks. This landscape is then flooded from the given *markers*, until separate<br>
flood basins meet at the peaks. Each distinct basin then forms a different<br>
image segment. [4]_<br>
As with SLIC, there is an additional *compactness* argument that makes it<br>
harder for markers to flood faraway pixels. This makes the watershed regions<br>
more regularly shaped. [5]_<br>
.. [4] https://en.wikipedia.org/wiki/Watershed_%28image_processing%29<br>
.. [5] Peer Neubert & Peter Protzel (2014). Compact Watershed and<br>
       Preemptive SLIC: On Improving Trade-offs of Superpixel Segmentation<br>
       Algorithms. ICPR 2014, pp 996-1001. :DOI:`10.1109/ICPR.2014.181`<br>
       https://www.tu-chemnitz.de/etit/proaut/publications/cws_pSLIC_ICPR.pdf<br>


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from skimage.data import astronaut
from skimage.color import rgb2gray
from skimage.filters import sobel
from skimage.segmentation import felzenszwalb, slic, quickshift, watershed
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float

In [ ]:
img = img_as_float(astronaut()[::2, ::2])

In [ ]:
segments_fz = felzenszwalb(img, scale=100, sigma=0.5, min_size=50)
segments_slic = slic(img, n_segments=250, compactness=10, sigma=1,
                     start_label=1)
segments_quick = quickshift(img, kernel_size=3, max_dist=6, ratio=0.5)
gradient = sobel(rgb2gray(img))
segments_watershed = watershed(gradient, markers=250, compactness=0.001)

In [ ]:
print(f"Felzenszwalb number of segments: {len(np.unique(segments_fz))}")
print(f"SLIC number of segments: {len(np.unique(segments_slic))}")
print(f"Quickshift number of segments: {len(np.unique(segments_quick))}")

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 10), sharex=True, sharey=True)

In [ ]:
ax[0, 0].imshow(mark_boundaries(img, segments_fz))
ax[0, 0].set_title("Felzenszwalbs's method")
ax[0, 1].imshow(mark_boundaries(img, segments_slic))
ax[0, 1].set_title('SLIC')
ax[1, 0].imshow(mark_boundaries(img, segments_quick))
ax[1, 0].set_title('Quickshift')
ax[1, 1].imshow(mark_boundaries(img, segments_watershed))
ax[1, 1].set_title('Compact watershed')

In [ ]:
for a in ax.ravel():
    a.set_axis_off()

In [ ]:
plt.tight_layout()
plt.show()